<a href="https://colab.research.google.com/github/Anonymous143w/New-TP/blob/main/Mediacal_cost_Predication/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

import os
import yaml
import logging
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Create project directory structure in Google Drive
BASE_DIR = '/content/drive/MyDrive/Medical Cost/'
DIRS = [
    '',
    'data/raw',
    'data/processed',
    'models',
    'config',
    'logs'
]

for dir_path in DIRS:
    full_path = os.path.join(BASE_DIR, dir_path)
    os.makedirs(full_path, exist_ok=True)

In [40]:
# Create config file
config = {
    'data': {
        'raw_data_path': os.path.join(BASE_DIR, 'data/raw/insurance.csv'),
        'processed_data_path': os.path.join(BASE_DIR, 'data/processed/insurance_processed.csv'),
        'test_size': 0.2,
        'random_state': 42
    },
    'model': {
        'type': 'polynomial',
        'degree': 2,
        'output_path': os.path.join(BASE_DIR, 'models/medical_cost_model.pkl')
    },
    'mlflow': {
        'tracking_uri': os.path.join(BASE_DIR, 'mlruns'),
        'experiment_name': 'medical_cost_prediction'
    }
}

In [ ]:
# Save config
with open(os.path.join(BASE_DIR, 'config/config.yaml'), 'w') as f:
    yaml.dump(config, f)

print("Project structure created in Google Drive!")

Project structure created in Google Drive!


In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.6/565.6 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.7 MB/s eta 0:00:00


In [54]:
!pip install uvicorn

In [ ]:
!pip install pyyaml

In [ ]:
!pip install -U scikit-learn

In [75]:
#Data ingestion and preprocessing classes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def initiate_data_ingestion(self):
        logging.info("Initiating data ingestion")
        try:
            from google.colab import files
            print("Please upload the insurance.csv file when prompted")
            uploaded = files.upload()

            # Save uploaded file to Google Drive
            with open(self.config['data']['raw_data_path'], 'wb') as f:
                f.write(uploaded[list(uploaded.keys())[0]])

            df = pd.read_csv(self.config['data']['raw_data_path'])

            train_set, test_set = train_test_split(
                df,
                test_size=self.config['data']['test_size'],
                random_state=self.config['data']['random_state']
            )

            return train_set, test_set

        except Exception as e:
            logging.error(f"Exception occurred in data ingestion: {e}")
            raise e

class DataPreprocessor:
    def __init__(self, config):
        self.config = config
        self.le = LabelEncoder()

    def transform_features(self, df):
        df_processed = df.copy()

        # Encode categorical variables
        categorical_cols = ['sex', 'smoker', 'region']
        for col in categorical_cols:
            df_processed[col] = self.le.fit_transform(df_processed[col])

        # Drop the 'charges' column only if it exists (during training)
        if 'charges' in df_processed.columns:
            X = df_processed.drop(['charges'], axis=1)
            y = df_processed['charges']
        else:
            X = df_processed
            y = None

        # Apply polynomial features if specified
        if self.config['model']['type'] == 'polynomial':
            poly = PolynomialFeatures(degree=self.config['model']['degree'])
            X = poly.fit_transform(X)

        return X, y


In [76]:
#Model training with MLflow
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import joblib

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        self.model = LinearRegression()

    def train(self, X_train, y_train, X_test, y_test):
        mlflow.set_tracking_uri(self.config['mlflow']['tracking_uri'])
        mlflow.set_experiment(self.config['mlflow']['experiment_name'])

        with mlflow.start_run():
            self.model.fit(X_train, y_train)

            # Make predictions
            y_train_pred = self.model.predict(X_train)
            y_test_pred = self.model.predict(X_test)

            # Calculate metrics
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

            # Log metrics
            mlflow.log_metric("train_r2", train_r2)
            mlflow.log_metric("test_r2", test_r2)
            mlflow.log_metric("test_rmse", test_rmse)

            # Log model
            mlflow.sklearn.log_model(self.model, "model")

            # Save model to Google Drive
            joblib.dump(self.model, self.config['model']['output_path'])

            print(f"Training Results:")
            print(f"Train R2 Score: {train_r2:.4f}")
            print(f"Test R2 Score: {test_r2:.4f}")
            print(f"Test RMSE: {test_rmse:.4f}")

        return self.model

In [77]:
#Training Pipeline
class TrainingPipeline:
    def __init__(self, config_path):
        with open(config_path) as f:
            self.config = yaml.safe_load(f)

    def run_pipeline(self):
        try:
            # Data Ingestion
            data_ingestion = DataIngestion(self.config)
            train_data, test_data = data_ingestion.initiate_data_ingestion()
            print("Data ingestion completed")

            # Data Preprocessing
            preprocessor = DataPreprocessor(self.config)
            X_train, y_train = preprocessor.transform_features(train_data)
            X_test, y_test = preprocessor.transform_features(test_data)
            print("Data preprocessing completed")

            # Model Training
            trainer = ModelTrainer(self.config)
            model = trainer.train(X_train, y_train, X_test, y_test)
            print("Model training completed")

            return model

        except Exception as e:
            logging.error(f"Error in training pipeline: {e}")
            raise e

In [79]:
#Run the pipeline
config_path = os.path.join(BASE_DIR, 'config/config.yaml')
pipeline = TrainingPipeline(config_path)
model = pipeline.run_pipeline()

Please upload the insurance.csv file when prompted


Saving insurance.csv to insurance (2).csv
Data ingestion completed
Data preprocessing completed


2024/10/10 15:49:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Training Results:
Train R2 Score: 0.8374
Test R2 Score: 0.8649
Test RMSE: 4579.3788
Model training completed


In [80]:
# FastAPI implementation
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

In [81]:
class MedicalCostInput(BaseModel):
    age: int
    sex: str
    bmi: float
    children: int
    smoker: str
    region: str

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [83]:
# Load the saved model
model = joblib.load(config['model']['output_path'])
preprocessor = DataPreprocessor(config)

@app.post("/predict")
async def predict_cost(input_data: MedicalCostInput):
    try:
        # Convert input data to DataFrame
        input_df = pd.DataFrame([input_data.dict()])

        # Preprocess the input
        X, _ = preprocessor.transform_features(input_df)

        # Make prediction
        prediction = model.predict(X)[0]

        return {"predicted_cost": float(prediction)}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Enable nested asyncio
nest_asyncio.apply()

In [72]:
'''
# Start the FastAPI server
port = 7000  # Or any other unused port
print(f"Starting server on port {port}...")
uvicorn.run(app, port=port)
'''

Starting server on port 7000...


INFO:     Started server process [4560]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4560]


In [61]:
!pip install pyngrok


In [84]:
!ngrok authtoken 2nFgDi15pDxpx4xZ5yrYEF3NNGy_3Y3UURBaSQgF3C8Xd1pvq


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [85]:
from pyngrok import ngrok

# Start ngrok to tunnel to port 7000
public_url = ngrok.connect(8000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://f47b-34-106-196-67.ngrok-free.app" -> "http://localhost:8000"
